In [8]:
import sys
sys.path.append('/home/mhasan3/Desktop/WorkFolder/')

In [9]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
sns.set()

In [10]:
# import the necessary keras packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor, AspectAwarePreprocessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet, MiniGoogleNet
##
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import cifar10
from imutils import paths

In [11]:
args = {
    "output": f"/home/mhasan3/Desktop/WorkFolder/keras_dir/biblio2/chap6_ensembling/outputs/",
    "models": f"/home/mhasan3/Desktop/WorkFolder/keras_dir/biblio2/chap6_ensembling/models/",
    "num_models": 5
}

In [12]:
(trainx, trainy), (testx, testy) = cifar10.load_data()
trainx = trainx.astype("float") / 255.0
testx = testx.astype("float") / 255.0

# encode the labels
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)
unique_class_names = ['airplanes', 'automobile', 'bird', 'cat', 'deer',
                      'dog', 'frog', 'horse', 'ship', 'truck']

In [13]:
# construct the data augmentation
aug = ImageDataGenerator(rotation_range=10,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [14]:
total_epochs = 40
lr = 0.01

# loop over the no of models to train
for i in range(args["num_models"]):
    print(f'[INFO] training model {i+1}/{args["num_models"]}.....')
    opt = SGD(lr=lr, momentum=0.9, nesterov=True, decay=lr/total_epochs)
    model = MiniGoogleNet.build(width=32, height=32, depth=3, classes=len(unique_class_names))
    model.compile(loss="categorical_crossentropy", optimizer=opt,
                  metrics=["accuracy"])
    # train the model
    H = model.fit_generator(aug.flow(trainx, trainy, batch_size=64),
                            validation_data=(testx, testy),
                            epochs=total_epochs, verbose=1)
    #save the model
    p = [args["models"], f"model_{i}.hdf5"]
    model.save(os.path.sep.join(p))

    # evaluate the network
    print ( "[INFO] evaluating network..." )
    predictions = model.predict(testx, batch_size=64)
    report = classification_report(testy.argmax(axis=1 ),
    predictions.argmax(axis=1 ), target_names=unique_class_names)

    # save the classification rreport and plot
    p = [args["output"], f"model_{i}.txt"]
    f = open(os.path.sep.join(p), "w")
    f.write(report)
    f.close()


    # plot the performance
    epochs = range(1,total_epochs+1)
    loss = H.history['loss']
    accuracy = H.history['acc']
    val_loss = H.history['val_loss']
    val_accuracy = H.history['val_acc']
    plot_df = pd.DataFrame(data=np.c_[epochs,loss,accuracy,val_loss,val_accuracy],
                           columns=['epochs','loss', 'accuracy', 'val_loss', 'val_accuracy'])

    # do the actual plots
    sns.set(font_scale=1)
    f, ax = plt.subplots(1, 1, figsize=(15,8))
    sns.lineplot(data=plot_df, x='epochs', y='loss', ax=ax, label='train loss', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='accuracy', ax=ax, label='train accuracy', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='val_loss', ax=ax, label='val loss', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='val_accuracy', ax=ax, label='val_accuracy', linewidth=3)
    ax.set_ylabel('Loss or Accuracy')
    ax.set_xlabel('Epochs')
    plt.setp(ax.get_legend().get_texts(), fontsize='18'); # for legend text
    p = [args["output"], f"model_{i}.png"]      
    plt.savefig(os.path.sep.join(p))
    plt.close()

    print("=" * 50)

[INFO] training model 1/5.....
Epoch 1/40
782/782 [==============================] - 75s 96ms/step - loss: 1.4116 - acc: 0.4934 - val_loss: 1.1053 - val_acc: 0.6111
Epoch 2/40
782/782 [==============================] - 74s 95ms/step - loss: 1.0232 - acc: 0.6383 - val_loss: 1.2042 - val_acc: 0.5788
Epoch 3/40
782/782 [==============================] - 74s 95ms/step - loss: 0.8939 - acc: 0.6841 - val_loss: 1.0406 - val_acc: 0.6512
Epoch 4/40
782/782 [==============================] - 74s 95ms/step - loss: 0.8036 - acc: 0.7178 - val_loss: 0.9540 - val_acc: 0.6646
Epoch 5/40
782/782 [==============================] - 75s 95ms/step - loss: 0.7473 - acc: 0.7383 - val_loss: 0.7525 - val_acc: 0.7398
Epoch 6/40
782/782 [==============================] - 74s 95ms/step - loss: 0.7005 - acc: 0.7567 - val_loss: 0.6909 - val_acc: 0.7635
Epoch 7/40
782/782 [==============================] - 74s 94ms/step - loss: 0.6591 - acc: 0.7697 - val_loss: 1.0155 - val_acc: 0.6678
Epoch 8/40
782/782 [===========

Epoch 37/40
782/782 [==============================] - 75s 95ms/step - loss: 0.3167 - acc: 0.8899 - val_loss: 0.6191 - val_acc: 0.8133
Epoch 38/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3114 - acc: 0.8934 - val_loss: 0.4572 - val_acc: 0.8501
Epoch 39/40
782/782 [==============================] - 75s 95ms/step - loss: 0.3049 - acc: 0.8940 - val_loss: 0.4974 - val_acc: 0.8371
Epoch 40/40
782/782 [==============================] - 74s 95ms/step - loss: 0.3015 - acc: 0.8951 - val_loss: 0.5157 - val_acc: 0.8348
[INFO] evaluating network...
[INFO] training model 2/5.....
Epoch 1/40
782/782 [==============================] - 76s 98ms/step - loss: 1.4422 - acc: 0.4788 - val_loss: 1.1594 - val_acc: 0.5915
Epoch 2/40
782/782 [==============================] - 75s 96ms/step - loss: 1.0407 - acc: 0.6290 - val_loss: 0.9720 - val_acc: 0.6616
Epoch 3/40
782/782 [==============================] - 75s 96ms/step - loss: 0.9007 - acc: 0.6853 - val_loss: 0.8938 - val_acc: 0.685

Epoch 33/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3356 - acc: 0.8832 - val_loss: 0.5220 - val_acc: 0.8316
Epoch 34/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3300 - acc: 0.8860 - val_loss: 0.5088 - val_acc: 0.8322
Epoch 35/40
782/782 [==============================] - 74s 95ms/step - loss: 0.3286 - acc: 0.8855 - val_loss: 0.4783 - val_acc: 0.8490
Epoch 36/40
782/782 [==============================] - 74s 95ms/step - loss: 0.3265 - acc: 0.8869 - val_loss: 0.5916 - val_acc: 0.8160
Epoch 37/40
782/782 [==============================] - 74s 95ms/step - loss: 0.3167 - acc: 0.8909 - val_loss: 0.5123 - val_acc: 0.8348
Epoch 38/40
782/782 [==============================] - 75s 95ms/step - loss: 0.3152 - acc: 0.8914 - val_loss: 0.5090 - val_acc: 0.8383
Epoch 39/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3086 - acc: 0.8917 - val_loss: 0.4934 - val_acc: 0.8421
Epoch 40/40
782/782 [==============================] - 

Epoch 29/40
782/782 [==============================] - 75s 95ms/step - loss: 0.3620 - acc: 0.8745 - val_loss: 0.4551 - val_acc: 0.8476
Epoch 30/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3555 - acc: 0.8779 - val_loss: 0.5508 - val_acc: 0.8185
Epoch 31/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3525 - acc: 0.8786 - val_loss: 0.4823 - val_acc: 0.8402
Epoch 32/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3463 - acc: 0.8819 - val_loss: 0.4707 - val_acc: 0.8454
Epoch 33/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3387 - acc: 0.8825 - val_loss: 0.4557 - val_acc: 0.8513
Epoch 34/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3370 - acc: 0.8832 - val_loss: 0.4345 - val_acc: 0.8553
Epoch 35/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3248 - acc: 0.8875 - val_loss: 0.5541 - val_acc: 0.8241
Epoch 36/40
782/782 [==============================] - 

Epoch 25/40
782/782 [==============================] - 74s 95ms/step - loss: 0.3955 - acc: 0.8644 - val_loss: 0.4948 - val_acc: 0.8326
Epoch 26/40
782/782 [==============================] - 74s 95ms/step - loss: 0.3876 - acc: 0.8675 - val_loss: 0.5839 - val_acc: 0.8089
Epoch 27/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3804 - acc: 0.8692 - val_loss: 0.6380 - val_acc: 0.7999
Epoch 28/40
782/782 [==============================] - 74s 95ms/step - loss: 0.3749 - acc: 0.8714 - val_loss: 0.6710 - val_acc: 0.7807
Epoch 29/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3678 - acc: 0.8726 - val_loss: 0.6475 - val_acc: 0.7912
Epoch 30/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3643 - acc: 0.8734 - val_loss: 0.4144 - val_acc: 0.8625
Epoch 31/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3539 - acc: 0.8775 - val_loss: 0.4425 - val_acc: 0.8514
Epoch 32/40
782/782 [==============================] - 

Epoch 21/40
782/782 [==============================] - 75s 96ms/step - loss: 0.4292 - acc: 0.8512 - val_loss: 0.4969 - val_acc: 0.8325
Epoch 22/40
782/782 [==============================] - 75s 96ms/step - loss: 0.4159 - acc: 0.8558 - val_loss: 0.7984 - val_acc: 0.7540
Epoch 23/40
782/782 [==============================] - 75s 95ms/step - loss: 0.4105 - acc: 0.8576 - val_loss: 0.5522 - val_acc: 0.8168
Epoch 24/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3991 - acc: 0.8629 - val_loss: 0.5694 - val_acc: 0.8172
Epoch 25/40
782/782 [==============================] - 75s 95ms/step - loss: 0.3931 - acc: 0.8641 - val_loss: 0.5361 - val_acc: 0.8163
Epoch 26/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3855 - acc: 0.8681 - val_loss: 0.5762 - val_acc: 0.8096
Epoch 27/40
782/782 [==============================] - 75s 96ms/step - loss: 0.3784 - acc: 0.8683 - val_loss: 0.6353 - val_acc: 0.7977
Epoch 28/40
782/782 [==============================] - 